In [1]:
import ee
import geemap

from src.utils.datautils import monitor_task

In [2]:
# ! earthengine create folder projects/nb-lidar/assets/exp3

# Experiment 3:
3 Class Problem: Non Wetland, Water, Wetland
Remove all Auxiliary Datasets i.e. Fourier Transform and Alos Palsar form the stack

stack will contain DataCube(Sentinel-2) and Sentinel - 1 compoistes for now + elevation prodcuts - elevation band

Training data will has been pre generated and saved in the form gee table assets, features will be extracted from the stack and used to train the model

In [3]:
ee.Initialize(project="nb-lidar")

In [4]:
# DEFINE CONSTANTS
folder = "exp3"
aoi_id = "projects/nb-lidar/assets/aoi_nb_south"
terrain_type = "srtm"

In [5]:
aoi = ee.FeatureCollection(aoi_id).geometry()

In [6]:
from src.helpers.rsd import (
    Sentinel1Collection,
    DataCubeCollection,
    TerrrainProdcutLayersFactory,
)


def fetch_and_process_s1(aoi, doy_start, doy_end) -> ee.Image:
    return (
        Sentinel1Collection()
        .filterBounds(aoi)
        .filterDate("2019", "2020")
        .filterDayOfYear(doy_start, doy_end)
        .filterDV()
        .filterIWMode()
        .applyEdgeMask()
        .median()
        .convolve(ee.Kernel.square(1))
        .select("V.*")
    )


def fetch_and_process_datacube(aoi) -> ee.Image:
    return (
        DataCubeCollection()
        .filterBounds(aoi)
        .mosaic()
        .select(".*_b[0][2-8].*|.*b[1][1-2].*")
    )


def fetch_ta(terrain_type: str):
    return (
        TerrrainProdcutLayersFactory()
        .get_layer(terrain_type)
        .select(ee.List.sequence(1, 5))
    )


def make_stack(aoi):
    # fetch and process data
    s1_es = fetch_and_process_s1(aoi, 79, 170)
    s1_ls = fetch_and_process_s1(aoi, 171, 265)
    dc = fetch_and_process_datacube(aoi)
    ta = fetch_ta("dtm")

    return ee.Image.cat(s1_es, s1_ls, dc, ta)

In [7]:
# build our features

import ee.batch

dest_id = f"projects/nb-lidar/assets/{folder}/{terrain_type}_features"

training_id = "projects/nb-lidar/assets/nb_south_3_class_bal"
stack = make_stack(aoi)
features = ee.FeatureCollection(training_id)
extracted_features = stack.sampleRegions(
    collection=features, tileScale=16, scale=10, geometries=True
)

# export to asset
task = ee.batch.Export.table.toAsset(
    collection=extracted_features, assetId=dest_id, description=""
)

task.start()
print(f"Exporting: {dest_id}")
status_code = monitor_task(task)

ee.Reset()

Exporting: projects/nb-lidar/assets/exp3/srtm_features


In [8]:
# train the smile Random Forest Model
from src.helpers.classifier import SmileRandomForest
from src.utils.datautils import confusion_matrix_to_drive
import ee

ee.Initialize(project="nb-lidar")

extracted_features = ee.FeatureCollection(extracted_features)

# spit the dataset into train and test
train = extracted_features.filter("random < 0.7")
test = extracted_features.filter("random >= 0.7")

rfm = SmileRandomForest()
rfm.train(
    features=train, class_property="class_name", input_properties=stack.bandNames()
)


# assess the model
order = test.aggregate_array("class_name").distinct()
cfm = rfm.error_matrix(test=test, order=order)

# save outputs
drive_folder_name = f"nb_lidar_{folder}"
confusion_matrix_to_drive(
    cfm, folder=drive_folder_name, name=f"{terrain_type}_confusion_matrix"
)
model_id = f"projects/nb-lidar/assets/{folder}/{terrain_type}_model"
rfm_task = rfm.save(model_id)

print(f"Exporting Model: {model_id}")
monitor_task(rfm_task)
ee.Reset()

Exporting Model: projects/nb-lidar/assets/exp3/srtm_model


In [9]:
# make predictions with the model
ee.Initialize(project="nb-lidar")

rfm = SmileRandomForest().load(model_id)
predict = rfm.predict(stack)

In [10]:
# visualize

Map = geemap.Map()

Map.addLayer(predict, {"min": 1, "max": 3}, "Prediction")
Map.centerObject(aoi, 8)
Map.addLayerControl()

Map

Map(center=[45.74809742262684, -65.99999969409251], controls=(WidgetControl(options=['position', 'transparent_…

In [11]:
# Export Results to cloud storage
from src.utils.datautils import classification_to_cloud_stroage

file_name_prefix = f"nb_lidar/{folder}/prediction/{terrain_type}/{terrain_type}-"

prediction_task = classification_to_cloud_stroage(
    image=predict,
    file_name=file_name_prefix,
    bucket="eerfpl-exports",
    region=aoi,
    start=False,
)

In [12]:
prediction_task.start()